In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
import logging
logging.basicConfig(level=logging.INFO)

try:
    spark = SparkSession.builder \
        .appName("Expedia Lodging Summary and Guest-Reviews") \
        .master("local[*]") \
        .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2") \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.local.type", "hadoop") \
        .config("spark.sql.catalog.local.warehouse", "summary-warehouse") \
        .getOrCreate()
    print("Spark Session created successfully")
except Exception as e:
    logging.error("Error creating Spark Session", exc_info=True)

spark

25/01/10 16:47:51 WARN Utils: Your hostname, w3e101 resolves to a loopback address: 127.0.1.1; using 192.168.0.231 instead (on interface enp2s0)
25/01/10 16:47:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/w3e101/Downloads/asset/summary/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/w3e101/.ivy2/cache
The jars for the packages stored in: /home/w3e101/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-459c52a4-924a-470d-86d1-514a8cd0bd27;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.2 in central
:: resolution report :: resolve 581ms :: artifacts dl 13ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.s

Spark Session created successfully


In [4]:
try:
    guest_review_df = spark.read.json("input/expedia-lodging-guestreviews-1-all.jsonl")
    summary_df = spark.read.json("input/expedia-lodging-summary-en_us-1-all.jsonl")

    print("Successfully created the dataframes")

except Exception as e:
    logging.error("Error reading data from JSON files", exc_info=True)

Successfully created the dataframes


In [5]:
guest_review_df.show(5)
summary_df.show(50)

+-------------+--------------------+--------------------+--------------------+--------------------+
|      country|         guestRating|        guestReviews|          propertyId|        propertyType|
+-------------+--------------------+--------------------+--------------------+--------------------+
|United States|{{3.375, 8}, {6.8...|[{NULL, 2023-01-0...|    {5624, 122257, }|          {1, Hotel}|
|       Canada|{{4.7910447, 67},...|[{2023-03-17, 202...|{33560494, 107493...|{17, Private vaca...|
|United States|{{4.8187494, 160}...|[{2023-03-12, 202...|{33560486, 107493...|{17, Private vaca...|
|      Austria|{{4.8461537, 13},...|                NULL|{25171841, 806498...|     {16, Apartment}|
|        Italy|{{4.17668, 283}, ...|[{2019-05-19, 201...|    {5540, 178103, }|          {1, Hotel}|
+-------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

+--------------------+---------------+--------------------+----------------

In [6]:
guest_review_df.count()


998230

In [7]:
summary_df.count()

1000000

In [8]:
guest_review_df.printSchema()

root
 |-- country: string (nullable = true)
 |-- guestRating: struct (nullable = true)
 |    |-- expedia: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |    |-- reviewCount: string (nullable = true)
 |    |-- hcom: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |    |-- reviewCount: string (nullable = true)
 |    |-- vrbo: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |    |-- reviewCount: string (nullable = true)
 |-- guestReviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- checkinDate: string (nullable = true)
 |    |    |-- checkoutDate: string (nullable = true)
 |    |    |-- creationDate: string (nullable = true)
 |    |    |-- languageCode: string (nullable = true)
 |    |    |-- reviewBrand: string (nullable = true)
 |    |    |-- reviewText: string (nullable = true)
 |    |    |-- reviewerName: string (nullable = true)
 |    |   

In [9]:
guest_review_df.show(5, truncate= False)

+-------------+------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
summary_df.printSchema()

root
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- bookable: struct (nullable = true)
 |    |-- expedia: boolean (nullable = true)
 |    |-- hcom: boolean (nullable = true)
 |    |-- vrbo: boolean (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- geoLocation: struct (nullable = true)
 |    |-- latitude: string (nullable = true)
 |    |-- longitude: string (nullable = true)
 |    |-- obfuscated: boolean (nullable = true)
 |-- guestRating: struct (nullable = true)
 |    |-- expedia: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |-- hcom: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |-- vrbo: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |-- heroImage: string (nullable = true)
 |-- lastUpdated: string (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- propertyId: struct (nullable = true)

In [11]:
summary_df.columns

['address1',
 'address2',
 'bookable',
 'city',
 'country',
 'geoLocation',
 'guestRating',
 'heroImage',
 'lastUpdated',
 'postalCode',
 'propertyId',
 'propertyName',
 'propertyType',
 'province',
 'referencePrice',
 'starRating']

In [12]:
len(summary_df.columns)

16

In [14]:
summary_df.select("propertyId").printSchema()

root
 |-- propertyId: struct (nullable = true)
 |    |-- expedia: string (nullable = true)
 |    |-- hcom: string (nullable = true)
 |    |-- vrbo: string (nullable = true)



In [19]:
from pyspark.sql.functions import col, trim
# Filter rows where propertyId is not null
non_null_count = summary_df.filter(col("address2").isNotNull() & (trim(col("address2")) != "")).count()

# Check if non_null_count is zero
if non_null_count == 0:
    print(f"The column 'address2' is empty.")
else:
    print(f"The column 'address2' is not empty. it has value in {non_null_count} rows")

The column 'address2' is not empty. it has value in 41745 rows


In [20]:
from pyspark.sql.functions import col, trim
# Filter rows where propertyId is not null
non_null_count = summary_df.filter(col("address1").isNotNull() & (trim(col("address1")) != "")).count()

# Check if non_null_count is zero
if non_null_count == 0:
    print(f"The column 'address1' is empty.")
else:
    print(f"The column 'address1' is not empty. it has value in {non_null_count} rows")

The column 'address1' is not empty. it has value in 218086 rows


In [25]:
from pyspark.sql.functions import expr, explode

new_df = guest_review_df.select(explode(col("guestReviews")).alias("reviewerScore")) \
           .select(col("reviewerScore.reviewerScore"))

In [26]:
new_df.show(5)

+-------------+
|reviewerScore|
+-------------+
|            2|
|            2|
|            1|
|            1|
|            1|
+-------------+
only showing top 5 rows



In [27]:
guest_review_df.printSchema()

root
 |-- country: string (nullable = true)
 |-- guestRating: struct (nullable = true)
 |    |-- expedia: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |    |-- reviewCount: string (nullable = true)
 |    |-- hcom: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |    |-- reviewCount: string (nullable = true)
 |    |-- vrbo: struct (nullable = true)
 |    |    |-- avgRating: string (nullable = true)
 |    |    |-- reviewCount: string (nullable = true)
 |-- guestReviews: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- checkinDate: string (nullable = true)
 |    |    |-- checkoutDate: string (nullable = true)
 |    |    |-- creationDate: string (nullable = true)
 |    |    |-- languageCode: string (nullable = true)
 |    |    |-- reviewBrand: string (nullable = true)
 |    |    |-- reviewText: string (nullable = true)
 |    |    |-- reviewerName: string (nullable = true)
 |    |   